In [1]:
import base64
import getpass
import json

import requests

In [3]:
CAS = 'http://localhost:7000'
RM = 'http://localhost:8002'
AR = 'http://localhost:8003'

## Get Token from Auth Service

Other services require tokens, the auth service is the only one that deals with the password.

In [2]:
username = ['alice', 'bob', 'cindy', 'dave'][0]
password = username.upper()

In [4]:
response = requests.post(CAS + '/auth/tokens', json={'username': username, 'password': password})
token = response.json()['token']

## Define a Site

The Sites provide a reference for Implementations and Credentials

In [5]:
response = requests.get(AR + '/sites/')
assert response.status_code == 200
sites = response.json()

In [6]:
response = requests.post(AR + '/sites/', 
    headers={'Dibbs-Authorization': token},
    json={
        'name': 'roger',
        'type': 'openstack',
        'api_url': 'http://roger-openstack.ncsa.illinois.edu:5000/v2.0',
    },
)
assert response.status_code == 201
site = response.json()

## Define an Appliance

An Appliance is something that provides a consistent API for higher-level functions. In and of itself, this definition doesn't do anything.

In [7]:
apps = requests.get(AR + '/appliances/').json()

In [8]:
#requests.delete(AR + '/appliances/{}/'.format(apps[0]['id']),
#                headers={'Dibbs-Authorization': token},
#)

In [9]:
response = requests.post(AR + '/appliances/', 
    headers={'Dibbs-Authorization': token},
    json={
        'name': 'hadoop',
        'description': 'Named after an Elephant from the San Diego zoo',
    },
)
assert response.status_code == 201
app = response.json()

## Define an Implementation

The implementation is a HOT (Heat orchestration template) that is set up for a specified site. Some might have additional options, for instance, Chameleon would require a reservation ID.

HOT outputs to-be-determined, but should at least include the public IP where the agent is reachable.

In [10]:
with open('appliances/hadoop-roger.yml', 'r') as f:
    template = f.read()

response = requests.post(AR + '/implementations/', 
    headers={'Dibbs-Authorization': token},
    json={
        'appliance': app['id'],
        'site': site['id'],
        'script': template
    },
)
assert response.status_code == 201
imp = response.json()

## Load (OpenStack) Credentials

They're stored in a super-secure (SARCASM WARNING) format: a base64-encoded JSON object. It's mostly to prevent easy screen reading of your password. The required fields in the object are `username`, `password`, and `project_name`.

In [12]:
import subprocess

def obfuscate(data):
    return base64.b64encode(json.dumps(data).encode('utf-8')).decode('utf-8')

roger_pass = subprocess.check_output('ssh roger "cat ~/.openstack"', shell=True, universal_newlines=True).strip()
creds = {
    'username': 'npt',
    'password': roger_pass,
    'project_name': 'DIBBs',
}
cred_string = obfuscate(creds)

In [13]:
response = requests.post(RM + '/credentials/',
    headers={'Dibbs-Authorization': token},
    json={
        'name': 'me-keys',
        'site': site['id'],
        'credentials': cred_string,
    }
)
assert response.status_code == 201
creds = response.json()

## Create a Cluster

(this is actually performed by get-creating the Resource, this is semi-internal)

In [14]:
# response = requests.post(RM + '/clusters/',
#     headers={'Dibbs-Authorization': token},
#     json={
#         'credential': creds['id'],
#         'implementation': imp['id'],
#     }
# )
# assert response.status_code == 201
# cluster = response.json()

In [15]:
# cluster

{'address': '',
 'credential': '0f94355d-6219-4111-94c7-93f2cd393cb2',
 'id': '2c99baa7-0d61-421f-9367-5d209839640b',
 'implementation': '5e238dcd-b9cc-41bc-9f38-378bc6bf490e',
 'remote_id': '1314d3c1-8c6e-4cdf-8ea8-2a83cbea4852',
 'remote_status': 'CREATE_IN_PROGRESS',
 'root_owner': 1,
 'status': 'BUILDING'}

In [28]:
# Deleting a cluster
response = requests.delete(RM + '/clusters/{}/'.format('13f3c9e8-8ff6-46fd-ba5a-b715f2dbce78'),
    headers={'Dibbs-Authorization': token},
)
response.status_code

204

In [27]:
requests.get(RM + '/clusters/').json()

[{'address': '141.142.170.218',
  'credential': '0f94355d-6219-4111-94c7-93f2cd393cb2',
  'id': '13f3c9e8-8ff6-46fd-ba5a-b715f2dbce78',
  'implementation': '5e238dcd-b9cc-41bc-9f38-378bc6bf490e',
  'remote_id': '0093d846-9993-4dd7-b1a4-e2e5a1b09a9b',
  'remote_status': 'CREATE_COMPLETE',
  'root_owner': 1,
  'status': 'READY'}]

## Get/Create a Resource

The hints are fairly heavy-handed for now, the credentials and implementation must be specified.

In [21]:
response = requests.post(RM + '/resources/', 
    headers={'Dibbs-Authorization': token},
    json={
        'hints': {
            'implementation': imp['id'],
            'credentials': creds['id'],
        },
    }
)
assert response.status_code == 201
resource = response.json()

In [22]:
resource

{'cluster': '13f3c9e8-8ff6-46fd-ba5a-b715f2dbce78',
 'hints': '{"implementation": "5e238dcd-b9cc-41bc-9f38-378bc6bf490e", "credentials": "0f94355d-6219-4111-94c7-93f2cd393cb2"}',
 'id': 'c2e90c19-9f89-46d9-9237-cb1252f2f50d',
 'password': '',
 'user': 1,
 'username': ''}

In [24]:
requests.get(RM + '/clusters/{}/'.format(resource['cluster'])).json()

{'address': '141.142.170.218',
 'credential': '0f94355d-6219-4111-94c7-93f2cd393cb2',
 'id': '13f3c9e8-8ff6-46fd-ba5a-b715f2dbce78',
 'implementation': '5e238dcd-b9cc-41bc-9f38-378bc6bf490e',
 'remote_id': '0093d846-9993-4dd7-b1a4-e2e5a1b09a9b',
 'remote_status': 'CREATE_COMPLETE',
 'root_owner': 1,
 'status': 'READY'}

In [25]:
response = requests.get(RM + '/resources/{}/'.format(resource['id']))
assert response.status_code == 200
response.json()

{'cluster': '13f3c9e8-8ff6-46fd-ba5a-b715f2dbce78',
 'hints': '{"implementation": "5e238dcd-b9cc-41bc-9f38-378bc6bf490e", "credentials": "0f94355d-6219-4111-94c7-93f2cd393cb2"}',
 'id': 'c2e90c19-9f89-46d9-9237-cb1252f2f50d',
 'password': '',
 'user': 1,
 'username': ''}